In [ ]:
# Load datasets

import os

dataset_dir = '/content/drive/MyDrive/face_features_88' # change to your dataset directory
image_paths = [os.path.join(dataset_dir, fname) for fname in os.listdir(dataset_dir) if fname.endswith('.jpg')]

In [ ]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 17.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [ ]:
# Extract ROI of face segment by YOLO(face detection) and OpenCV(ROI)

from ultralytics import YOLO
import mediapipe as mp
import pandas as pd

data = []
model = YOLO('yolov8n.pt')
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

In [ ]:
import numpy as np

def extract_landmarks(rgb_image):
    result = face_mesh.process(rgb_image)
    if result.multi_face_landmarks:
        print(f"Number of faces detected: {len(result.multi_face_landmarks)}")
        landmarks = result.multi_face_landmarks[0]
        return [(lm.x, lm.y, lm.z) for lm in landmarks.landmark]
    return None

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# 감지된 얼굴과 특징을 포함한 이미지를 저장할 리스트
result_images = []

for path in image_paths:
    print(image_paths)
    image = cv2.imread(path)  # 이미지 로드
    if image is None:
        print(f"Error: Failed to load image at {path}")
        continue  # 이미지 로드 실패 시 다음 이미지로 넘어감

    results = model(image)  # YOLO를 사용하여 얼굴 감지

    # 결과 리스트에 대해 반복하여 각 이미지의 얼굴에 대한 작업 수행
    for result in results:
        boxes = result.boxes

        for box in boxes:
            # 바운딩 박스 좌표 추출
            x1, y1, x2, y2 = int(box.xmin), int(box.ymin), int(box.xmax), int(box.ymax)

            # ROI 추출 및 바운딩 박스 그리기
            roi = image[y1:y2, x1:x2]
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)



            # box의 이름이 'face'인 경우 얼굴 랜드마크 추출
            if box.name == 'face':
                face_landmarks = extract_landmarks(roi)
            # box의 이름이 'eye', 'nose', 'mouth'인 경우 특징 랜드마크 추출
            elif box.name in ['eye', 'nose', 'mouth']:
                feature_landmarks = extract_landmarks(roi)
            else:
              print(box.name)
              if box.name == None:
                print("None")

            # 데이터 저장 (선택 사항)
            data.append({
                'image_path': path,
                'bounding_box': (x1, y1, x2, y2),
                'name': box.name,
                'landmarks': face_landmarks if box.name == 'face' else feature_landmarks
            })

    # 결과 이미지를 리스트에 추가
    result_images.append(image)

# 1, 2, 3번째 결과 이미지를 표시
for i in range(min(3, len(result_images))):
    cv2_imshow(result_images[i])

# 작업 완료 후 모든 윈도우 닫기
cv2.destroyAllWindows()


['/content/drive/MyDrive/face_features_88/face features_52.jpg', '/content/drive/MyDrive/face_features_88/humans faces_91.jpg', '/content/drive/MyDrive/face_features_88/face features_49.jpg', '/content/drive/MyDrive/face_features_88/face features_67.jpg', '/content/drive/MyDrive/face_features_88/face features_18.jpg', '/content/drive/MyDrive/face_features_88/face features_24.jpg', '/content/drive/MyDrive/face_features_88/face features_17.jpg', '/content/drive/MyDrive/face_features_88/humans faces_100.jpg', '/content/drive/MyDrive/face_features_88/face features_66.jpg', '/content/drive/MyDrive/face_features_88/face features_85.jpg', '/content/drive/MyDrive/face_features_88/humans faces_93.jpg', '/content/drive/MyDrive/face_features_88/face features_88.jpg', '/content/drive/MyDrive/face_features_88/face features_81.jpg', '/content/drive/MyDrive/face_features_88/face features_25.jpg', '/content/drive/MyDrive/face_features_88/face features_87.jpg', '/content/drive/MyDrive/face_features_88/

AttributeError: 'Boxes' object has no attribute 'xmin'. See valid attributes below.

    Manages detection boxes, providing easy access and manipulation of box coordinates, confidence scores, class
    identifiers, and optional tracking IDs. Supports multiple formats for box coordinates, including both absolute and
    normalized forms.

    Attributes:
        data (torch.Tensor): The raw tensor containing detection boxes and their associated data.
        orig_shape (tuple): The original image size as a tuple (height, width), used for normalization.
        is_track (bool): Indicates whether tracking IDs are included in the box data.

    Properties:
        xyxy (torch.Tensor | numpy.ndarray): Boxes in [x1, y1, x2, y2] format.
        conf (torch.Tensor | numpy.ndarray): Confidence scores for each box.
        cls (torch.Tensor | numpy.ndarray): Class labels for each box.
        id (torch.Tensor | numpy.ndarray, optional): Tracking IDs for each box, if available.
        xywh (torc

In [ ]:
def calculate_similarity(landmarks1, landmarks2):
    return distance.euclidean(np.array(landmarks1), np.array(landmarks2))